In [3]:
import pandas as pd

train_data_path = "E:/page ds/pages/ts_mixer/covTrain.csv" 
forecast_data_path = "E:/page ds/pages/ts_mixer/covForecast.csv" 

data_train = pd.read_csv(train_data_path)
forecast_data = pd.read_csv(forecast_data_path)

data_train.set_index(data_train)

In [17]:
mi = data_train['index'].max()
forecast_data['index'] = [i + mi for i in forecast_data['index']]

forecast_data = forecast_data.set_index('index')
data_train = data_train.set_index('index')

In [40]:
data_train = data_train.set_index('index')

In [42]:
forecast_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 713 to 726
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   przypadki   14 non-null     int64  
 1   Arima_lag1  14 non-null     float64
 2   Arima_bc    14 non-null     float64
 3   Arima_auto  14 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 560.0 bytes
